In [1]:
import numpy as np
import torch
import sys

In [2]:
def write_obj(vertices, output_obj_path='../outputs/output.obj'):
    with open(output_obj_path, 'w') as obj_file:
        for vertex in vertices:
            obj_file.write(f'v {vertex[0]} {vertex[1]} {vertex[2]}\n')

    print(f'OBJ file saved to {output_obj_path}')

In [3]:
train_data = np.load('../all_data/points_shapenet_32x32x32_train.npy')
batch_size = 8

sample_batch = train_data[:batch_size, :, :]
sample_batch = torch.tensor(sample_batch).float()
sample_batch.shape

torch.Size([8, 3000, 3])

In [7]:
sys.path.append('../code')
import model

DMC = model.RawDeepMarchingCube()

offset, topology, occupancy = DMC(sample_batch)

In [9]:
import torch.nn as nn

_fake_offset = torch.zeros_like(offset).sum(1).sum(1).sum(1)
_offset = offset.sum(1).sum(1).sum(1)

class MeshLoss(nn.modules.loss._Loss):
    def __init__(self):
        super(MeshLoss, self).__init__()

    def forward(self, input, target):
        return torch.nn.MSELoss()(input, target)
    
myLoss = MeshLoss()
loss_offset = myLoss(_fake_offset, _offset)
loss_offset.backward()